In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 
import datetime

In [ ]:
data =  pd.read_csv('DATA_FOR_TRAIN.csv')

In [ ]:
data = data.drop(columns='Unnamed: 0', axis = 0)
data.head(10)

,USER_ID,CATE_ID,RATING
0,0,867,1
1,0,244,2
2,0,148,1
3,0,745,1
4,1,170,1
5,1,173,1
6,2,373,3
7,2,126,2
8,3,751,1
9,4,691,1


In [ ]:
data1 = pd.DataFrame(columns=[ 'CATE_ID','USER_ID', 'RATING'])
data1['USER_ID'] = data['USER_ID']
data1['CATE_ID'] = data['CATE_ID']
data1['RATING'] = data['RATING']
data1.tail(10)

,CATE_ID,USER_ID,RATING
2124658,810,799483,5
2124659,761,799483,2
2124660,261,799483,5
2124661,19,799483,1
2124662,877,799483,2
2124663,260,799483,1
2124664,262,799483,4
2124665,7,799483,2
2124666,14,799483,1
2124667,765,799483,5


In [ ]:
data1.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2124668 entries, 0 to 2124667
Data columns (total 3 columns):
CATE_ID    int64
USER_ID    int64
RATING     int64
dtypes: int64(3)
memory usage: 48.6 MB


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(data1, test_size=0.3, random_state=68)

In [ ]:
len_Xtrain = len(X_train)
len_Xtrain
#len_us = len(user_unique)

1487267

In [ ]:
len(X_test)

637401

In [ ]:
X_test['CATE_ID'].max()

951

In [ ]:
X_train.head(10)

,CATE_ID,USER_ID,RATING
1654452,503,622374,1
2101528,773,790800,2
1885718,31,709005,3
1331044,761,500222,1
54639,785,20514,3
1777053,745,668202,1
2055698,360,773271,2
33492,260,12556,1
372680,632,139794,2
1971481,554,741026,1


In [ ]:
class CF(object):
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data 
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1
    
    def add(self, new_data):
        """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
    
    def normalize_Y(self):
        users = self.Y_data[:, 0] # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in xrange(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        eps = 1e-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
        return self.S
    
        
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again 
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()
        
    
    def pred(self, i):
        """ 
        predict k similarity for item i (normalized)
        """
        recommended_items = []

        for item in range (951):
          if self.S[i,item] > 0.005:
            recommended_items.append(item)

        return recommended_items
    
            
    
   


In [ ]:
Y_data = X_train.as_matrix()
Y_data

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[   503, 622374,      1],
       [   773, 790800,      2],
       [    31, 709005,      3],
       ...,
       [   553, 216132,      5],
       [   260, 633749,      1],
       [   433, 328277,      1]])

In [ ]:
rs = CF(Y_data, k = 10, uuCF = 0 )
rs.fit()

test = rs.similarity()

In [ ]:

for n in range (952):
  a = rs.pred( i = n )
  print a

[0, 1, 2, 3, 6, 7, 8, 10, 13, 14, 15, 16, 18, 19, 20, 26, 31, 48, 105, 120, 129, 130, 133, 140, 151, 154, 162, 198, 199, 227, 228, 229, 230, 231, 236, 237, 246, 247, 258, 259, 260, 261, 262, 263, 302, 345, 348, 375, 424, 432, 445, 460, 463, 464, 472, 474, 487, 498, 529, 539, 540, 548, 557, 558, 560, 561, 563, 586, 761, 762, 763, 764, 767, 784, 785, 787, 789, 805, 806, 809, 811, 826, 836, 845, 881, 885, 919, 933, 935, 945]
[0, 1, 2, 3, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 24, 28, 31, 38, 48, 53, 121, 125, 130, 145, 146, 155, 157, 198, 199, 227, 228, 230, 231, 236, 237, 238, 247, 258, 259, 260, 261, 262, 263, 264, 314, 344, 357, 375, 423, 432, 442, 452, 460, 463, 464, 465, 470, 474, 478, 479, 485, 486, 498, 502, 540, 557, 558, 560, 561, 563, 567, 568, 640, 709, 742, 756, 761, 763, 764, 767, 768, 769, 784, 785, 788, 791, 805, 806, 811, 861, 862, 880, 881, 885, 933]
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 20, 31, 32, 38, 42, 45, 48, 50, 52, 117, 118, 119, 12

In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rs, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.pred(53)
print(result)

[1, 6, 13, 16, 38, 45, 48, 49, 51, 52, 53, 58, 77, 104, 119, 129, 131, 140, 142, 144, 146, 154, 155, 157, 163, 200, 217, 223, 227, 230, 231, 262, 264, 265, 303, 343, 358, 374, 377, 435, 448, 450, 455, 458, 461, 470, 492, 494, 496, 514, 515, 524, 527, 529, 530, 550, 563, 600, 605, 615, 627, 630, 643, 648, 672, 673, 687, 761, 781, 785, 787, 791, 800, 805, 806, 812, 820, 826, 827, 842, 844, 858, 868]


In [ ]:
test

array([[1.        , 0.10875155, 0.02882378, ..., 0.        , 0.00256226,
        0.        ],
       [0.10875155, 1.        , 0.03672306, ..., 0.        , 0.        ,
        0.        ],
       [0.02882378, 0.03672306, 1.        , ..., 0.        , 0.00828207,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00256226, 0.        , 0.00828207, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])